In [56]:
import numpy as np
import matplotlib.pyplot as plt

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.callback import CallbackCollection
from pymoo.problems import get_problem
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.visualization.scatter import Scatter
from pymoo.problems.dynamic.df import DF1

from pymoo.visualization.video.callback_video import ObjectiveSpaceAnimation

https://eden.dei.uc.pt/~paquete/tsp

In [63]:
def parse_data(url):
    data = {
        'name': None,
        'n': None,
        'dist': None,
        'coords': None,
        'p': []
    }
    last_section = ""
    with open(url) as f:
        lines = f.readlines()
        # print(lines)
    for line in lines:
        elements = line.strip().split()
        section_type = elements[0]
        if section_type == 'NAME:':
            data['name'] = elements[1]
            last_section = section_type

        elif section_type.startswith("DIMENSION"):
            data['n'] = int(elements[-1])
            last_section = section_type

        elif section_type == "EDGE_WEIGHT_SECTION":
            last_section = section_type

        elif section_type in ["DISPLAY_DATA_SECTION", "NODE_COORD_SECTION"]:
            last_section = section_type

        elif section_type == "TOUR_SECTION" or section_type == "EOF":
            last_section = section_type
            if section_type == "EOF":
                break
        else:
            if last_section == "EDGE_WEIGHT_SECTION":
                elements = np.array(list(map(int, elements)))
                if len(elements) < 29:  # If the distance matrix is incomplete
                    elements = np.hstack((np.zeros(29 - len(elements)), elements))
                if 'dist' in data:
                    data['dist'] = np.vstack((data['dist'], elements)) if data['dist'] is not None else elements
                else:
                    data['dist'] = elements
            elif last_section in ["DISPLAY_DATA_SECTION", "NODE_COORD_SECTION"]:
                elements = list(map(float, elements[1:]))
                if 'coords' in data:
                    data['coords'] = np.vstack((data['coords'], np.array(elements))) if data['coords'] is not None else np.array(elements)
                else:
                    data['coords'] = np.array(elements)

            elif last_section == "TOUR_SECTION" and elements[0] != '-1':
                element = int(elements[0])
                data['p'].append(element - 1)
    return data

In [76]:
randomA100 = parse_data('./dataset/randomA100.tsp')
A = randomA100['dist']

In [73]:
class TSPProblem(Problem):
    def __init__(self, distances):
        num_cities = len(distances)
        super().__init__(n_var=num_cities, n_obj=1, n_constr=0, xl=0, xu=num_cities-1, elementwise_evaluation=True)
        self.distances = distances

    def _evaluate(self, x, out, *args, **kwargs):
        total_distance = 0
        for i in range(len(x) - 1):
            total_distance += self.distances[x[i], x[i + 1]]
        total_distance += self.distances[x[-1], x[0]]  # Return to the starting city
        out["F"] = total_distance
# distances = np.array(A)
tsp_problem = TSPProblem(A)

In [74]:
tsp_problem.distances

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,  811.],
       [   0.,    0.,    0., ...,    0.,    0., 4308.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0., 3214.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [75]:
algorithm = NSGA2(pop_size=100)
res = minimize(tsp_problem,
               algorithm,
               ('n_gen', 200),
               seed=1,
               verbose=False)

plot = Scatter()
plot.add(A.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="red")
plot.show()


IndexError: arrays used as indices must be of integer (or boolean) type